# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *
 

In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
sigma_yield = personal_data[0]
EModulus = personal_data[1]
nu = personal_data[2]
print(f"Your personal data is: sigma_yield = {sigma_yield}, EModulus = {EModulus}, nu = {nu}. Please verify.")

#Panel constants 
length = 750
width = 200


Your personal data is: sigma_yield = 490, EModulus = 65241.07, nu = 0.34. Please verify.


# Open stress values & properties of panels 

In [5]:
#Imoport panel stress file and reformat (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)
# paneldf = paneldf.drop(['sigmaZZ', 'sigmaXZ', 'sigmaYZ'],axis=1)
# paneldf = paneldf.reset_index()


########
# New version of importing panel stress file
########

paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#Import panel properties and reformat
panelPropertiesdf = pd.read_excel(f'../data/{name}/panel_properties.xlsx', index_col=0)
panelPropertiesdf = panelPropertiesdf.reset_index()
panelPropertiesdf = panelPropertiesdf.rename(columns={'elements': 'Element ID'})

paneldf = paneldf_v2

#print(paneldf.head(5))
print(paneldf_v2.head(5))
#print(panelPropertiesdf.head(5))

                  Load Case    sigmaXX    sigmaXY    sigmaYY Component Name
Element ID                                                                 
1           Subcase 1 (LC1) -65.783310  25.897854  12.579947         panel1
2           Subcase 1 (LC1) -81.867737  26.737642  20.594475         panel1
3           Subcase 1 (LC1) -66.729820  36.480675  20.465578         panel1
4           Subcase 1 (LC1) -96.377647  34.062004   2.383052         panel2
5           Subcase 1 (LC1) -88.315163  26.684858  25.010015         panel2


# Split up the loadcases 

In [6]:
loadCase1df = paneldf[paneldf['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = paneldf[paneldf['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = paneldf[paneldf['Load Case'] == 'Subcase 3 (LC3)']
# Add the data to load case 
loadCase1df = pd.merge(loadCase1df, panelPropertiesdf, on='Element ID')
loadCase2df = pd.merge(loadCase2df, panelPropertiesdf, on='Element ID')
loadCase3df = pd.merge(loadCase3df, panelPropertiesdf, on='Element ID')

#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name,thickness,mass
Element ID,,,,,,,
1,Subcase 1 (LC1),-65.783310,25.897854,12.579947,panel1,4,0.00054
2,Subcase 1 (LC1),-81.867737,26.737642,20.594475,panel1,4,0.00054
3,Subcase 1 (LC1),-66.729820,36.480675,20.465578,panel1,4,0.00054
4,Subcase 1 (LC1),-96.377647,34.062004,2.383052,panel2,4,0.00054
5,Subcase 1 (LC1),-88.315163,26.684858,25.010015,panel2,4,0.00054


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [7]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC1['length'] = [length]*len(panelLC1)
panelLC1['width'] = [width]*len(panelLC1)
panelLC1

#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC2['length'] = [length]*len(panelLC2)
panelLC2['width'] = [width]*len(panelLC2)
panelLC2

#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
    'thickness':'median'
})
panelLC3['length'] = [length]*len(panelLC3)
panelLC3['width'] = [width]*len(panelLC3)
panelLC3

panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width
Component Name,,,,,,
panel1,-71.460289,17.880000,29.705390,4.0,750,200
panel10,-71.460289,17.880000,29.705390,4.0,750,200
panel2,-93.234734,17.879999,29.610400,4.0,750,200
panel3,-93.370883,17.880000,29.611538,4.0,750,200
panel4,-92.718221,17.880000,30.064287,4.0,750,200
panel5,-92.223958,17.880000,52.546577,4.0,750,200
panel6,-92.223958,17.880000,52.546577,4.0,750,200
panel7,-92.718221,17.880000,30.064287,4.0,750,200
panel8,-93.370883,17.880000,29.611538,4.0,750,200


# Now we can apply the functions to the load cases 

In [8]:
panelLC1[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC2[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC3[['k_shear', 'k_biaxial', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, EModulus=EModulus, nu=nu, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,thickness,length,width,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,,,,
panel1,-71.460289,17.880000,29.705390,4.0,750,200,5.624444,5.051196,1.019408
panel10,-71.460289,17.880000,29.705390,4.0,750,200,5.624444,5.051196,1.019408
panel2,-93.234734,17.879999,29.610400,4.0,750,200,5.624444,4.830947,0.769983
panel3,-93.370883,17.880000,29.611538,4.0,750,200,5.624444,4.829519,0.768739
panel4,-92.718221,17.880000,30.064287,4.0,750,200,5.624444,4.836409,0.772766
panel5,-92.223958,17.880000,52.546577,4.0,750,200,5.624444,4.841704,0.661931
panel6,-92.223958,17.880000,52.546577,4.0,750,200,5.624444,4.841704,0.661931
panel7,-92.718221,17.880000,30.064287,4.0,750,200,5.624444,4.836409,0.772766
panel8,-93.370883,17.880000,29.611538,4.0,750,200,5.624444,4.829519,0.768739


# Combining back into 1 Data Frame to output 

In [9]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length','thickness', 'width'], axis=1)
panelLC2 = panelLC2.drop(['length','thickness', 'width'], axis=1)
panelLC3 = panelLC3.drop(['length','thickness', 'width'], axis=1)
panelLC1.head(3)


,sigmaXX,sigmaYY,sigmaXY,k_shear,k_biaxial,Reserve Factor
Component Name,,,,,,
panel1,-71.460289,17.880000,29.70539,5.624444,5.051196,1.019408
panel10,-71.460289,17.880000,29.70539,5.624444,5.051196,1.019408
panel2,-93.234734,17.879999,29.61040,5.624444,4.830947,0.769983


In [10]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'k_shear':'k_shearLC1', 'k_biaxial':'k_biaxialLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'k_shear':'k_shearLC2', 'k_biaxial':'k_biaxialLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'k_shear':'k_shearLC3', 'k_biaxial':'k_biaxialLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF
Component Name,,,,,,
panel1,-71.460289,17.880000,29.705390,5.624444,5.051196,1.019408
panel10,-71.460289,17.880000,29.705390,5.624444,5.051196,1.019408
panel2,-93.234734,17.879999,29.610400,5.624444,4.830947,0.769983
panel3,-93.370883,17.880000,29.611538,5.624444,4.829519,0.768739
panel4,-92.718221,17.880000,30.064287,5.624444,4.836409,0.772766
panel5,-92.223958,17.880000,52.546577,5.624444,4.841704,0.661931
panel6,-92.223958,17.880000,52.546577,5.624444,4.841704,0.661931
panel7,-92.718221,17.880000,30.064287,5.624444,4.836409,0.772766
panel8,-93.370883,17.880000,29.611538,5.624444,4.829519,0.768739


In [11]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-71.460289,17.880000,29.705390,5.624444,5.051196,1.019408,-73.089521,-17.880000,29.705391,5.624444,3.040362,0.627986,-23.740163,5.940000,118.422834,5.624444,5.051196,0.504028
panel10,-71.460289,17.880000,29.705390,5.624444,5.051196,1.019408,-73.089521,-17.880000,29.705391,5.624444,3.040362,0.627986,-23.740163,5.940000,118.422834,5.624444,5.051196,0.504028
panel2,-93.234734,17.879999,29.610400,5.624444,4.830947,0.769983,-92.500331,-17.879999,29.610399,5.624444,3.227662,0.532710,-30.973955,5.940001,118.044144,5.624444,4.830947,0.481001
panel3,-93.370883,17.880000,29.611538,5.624444,4.829519,0.768739,-93.896586,-17.880000,29.611539,5.624444,3.238834,0.526944,-31.019186,5.940001,118.048681,5.624444,4.829519,0.480810
panel4,-92.718221,17.880000,30.064287,5.624444,4.836409,0.772766,-93.708064,-17.880000,30.064286,5.624444,3.237341,0.526807,-30.802361,5.940001,119.853600,5.624444,4.836409,0.469850
panel5,-92.223958,17.880000,52.546577,5.624444,4.841704,0.661931,-93.257683,-17.880000,52.546577,5.624444,3.233754,0.472615,-30.638160,5.939999,209.480988,5.624444,4.841704,0.175737
panel6,-92.223958,17.880000,52.546577,5.624444,4.841704,0.661931,-93.257683,-17.880000,52.546577,5.624444,3.233754,0.472615,-30.638160,5.939999,209.480988,5.624444,4.841704,0.175737
panel7,-92.718221,17.880000,30.064287,5.624444,4.836409,0.772766,-93.708064,-17.880000,30.064286,5.624444,3.237341,0.526807,-30.802361,5.940001,119.853600,5.624444,4.836409,0.469850
panel8,-93.370883,17.880000,29.611538,5.624444,4.829519,0.768739,-93.896586,-17.880000,29.611539,5.624444,3.238834,0.526944,-31.019186,5.940001,118.048681,5.624444,4.829519,0.480810


# ROUND & Output the file 

In [12]:
outputDf = outputDf.round(rounding_digits)
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')
outputDf.head(10)

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,k_shearLC1,k_biaxialLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,k_shearLC2,k_biaxialLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,k_shearLC3,k_biaxialLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-71.4603,17.88,29.7054,5.6244,5.0512,1.0194,-73.0895,-17.88,29.7054,5.6244,3.0404,0.6280,-23.7402,5.94,118.4228,5.6244,5.0512,0.5040
panel10,-71.4603,17.88,29.7054,5.6244,5.0512,1.0194,-73.0895,-17.88,29.7054,5.6244,3.0404,0.6280,-23.7402,5.94,118.4228,5.6244,5.0512,0.5040
panel2,-93.2347,17.88,29.6104,5.6244,4.8309,0.7700,-92.5003,-17.88,29.6104,5.6244,3.2277,0.5327,-30.9740,5.94,118.0441,5.6244,4.8309,0.4810
panel3,-93.3709,17.88,29.6115,5.6244,4.8295,0.7687,-93.8966,-17.88,29.6115,5.6244,3.2388,0.5269,-31.0192,5.94,118.0487,5.6244,4.8295,0.4808
panel4,-92.7182,17.88,30.0643,5.6244,4.8364,0.7728,-93.7081,-17.88,30.0643,5.6244,3.2373,0.5268,-30.8024,5.94,119.8536,5.6244,4.8364,0.4698
panel5,-92.2240,17.88,52.5466,5.6244,4.8417,0.6619,-93.2577,-17.88,52.5466,5.6244,3.2338,0.4726,-30.6382,5.94,209.4810,5.6244,4.8417,0.1757
panel6,-92.2240,17.88,52.5466,5.6244,4.8417,0.6619,-93.2577,-17.88,52.5466,5.6244,3.2338,0.4726,-30.6382,5.94,209.4810,5.6244,4.8417,0.1757
panel7,-92.7182,17.88,30.0643,5.6244,4.8364,0.7728,-93.7081,-17.88,30.0643,5.6244,3.2373,0.5268,-30.8024,5.94,119.8536,5.6244,4.8364,0.4698
panel8,-93.3709,17.88,29.6115,5.6244,4.8295,0.7687,-93.8966,-17.88,29.6115,5.6244,3.2388,0.5269,-31.0192,5.94,118.0487,5.6244,4.8295,0.4808
